In [1]:
import pandas as pd
import numpy as np

# Loading The Dataset

In [2]:
data = pd.read_csv('your_dataset.csv')

# Preparing data:
1. Seperating features and labels
2. Normalizing the dataset

In [3]:
def DataNormalize(inputData):
    mean = np.mean(inputData, axis = 0)
    std = np.std(inputData, axis = 0)
    normalizedData = (inputData - mean) / std
    
    return normalizedData

def DataShuffle(dataSize):
    indices = np.arange(dataSize)
    np.random.shuffle(indices)
    
    return indices

def DataSplit(inputData, outputData):
    # Define the split ratio
    trainRatio = 0.8
    trainSize = int(len(inputData) * trainRatio)
    
    # Split the data
    trainData = inputData[:trainSize]
    trainLabel = outputData[:trainSize]
    
    testData = inputData[trainSize:]
    testLabel = outputData[trainSize:]
    
    return trainData, trainLabel, testData, testLabel

def zeroOneLoss(trueLabels, predLabels):
    return np.sum(trueLabels != predLabels) / len(trueLabels)

In [4]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values

X_normalized = DataNormalize(X)

# Shuffle the data
indices = DataShuffle(X.shape[0])
X_normalized = X_normalized[indices]
Y = Y[indices]
    
    
# Split the data
trainData, trainLabel, testData, testLabel = DataSplit(X_normalized, Y)

# Algorithm Implementation

In [5]:
class KernelPegasosSVM:
    def __init__(self, kernel='gaussian', _lambda=1e-4, gamma=0.1, degree=3, coef0=1, max_iter=1000):
        self.kernel = kernel
        self.max_iter = max_iter
        self._lambda = _lambda
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.max_iter = max_iter
        self.alphas = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.X_train = None
        
    def _kernel_function(self, x1, x2):
        if self.kernel == 'gaussian':
            return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)
        elif self.kernel == 'polynomial':
            return (self.gamma * np.dot(x1, x2) + self.coef0) ** self.degree
        else:
            raise ValueError(f"Unknown kernel: {self.kernel}")
            
    def _compute_kernel_matrix(self, X):
        n_samples = X.shape[0]
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i, j] = self._kernel_function(X[i], X[j])
        return K
    
    def fit(self, X, Y):
        n_samples, n_features = X.shape
        self.alphas = np.zeros(n_samples)
        self.X_train = X
        K = self._compute_kernel_matrix(X)
        
        for t in range(1, self.max_iter + 1):
            eta_t = 1 / (self._lambda * t)
            i = np.random.randint(0, n_samples)
            condition = Y[i] * np.sum(self.alphas * Y * K[:, i]) < 1
            if condition:
                self.alphas[i] = (1 - eta_t * self._lambda) * self.alphas[i] + eta_t
            else:
                self.alphas[i] = (1 - eta_t * self._lambda) * self.alphas[i]
        
        support_vector_indices = np.where(self.alphas > 1e-5)[0]
        self.support_vectors = X[support_vector_indices]
        self.support_vector_labels = Y[support_vector_indices]
        self.alphas = self.alphas[support_vector_indices]
    
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        
        for i in range(X.shape[0]):
            kernel_sum = 0
            for alpha, sv_y, sv in zip(self.alphas, self.support_vector_labels, self.support_vectors):
                kernel_sum += alpha * sv_y * self._kernel_function(sv, X[i])
            pred[i] = np.sign(kernel_sum)
        
        return pred
            

In [6]:
# Grid Search using Cross Validateion
def crossValScore(X, y, params, k = 5):
    fold_size = len(X) // k
    accuracies = []
    
    for i in range(k):
        
        X_test = X[i * fold_size:(i + 1) * fold_size]
        y_test = y[i * fold_size:(i + 1) * fold_size]
        X_train = np.concatenate((X[:i * fold_size], X[(i + 1) * fold_size:]), axis=0)
        y_train = np.concatenate((y[:i * fold_size], y[(i + 1) * fold_size:]), axis=0)

        kernelPegasosSVM = KernelPegasosSVM(kernel = params['kernel'], _lambda = params['_lambda'], gamma = params['gamma'], degree =  params['degree'], coef0 = params['coef0'], max_iter =  params['max_iter'])
        
        kernelPegasosSVM.fit(X_train, y_train)
        
        predictions = kernelPegasosSVM.predict(X_test)
        accuracy = zeroOneLoss(predictions, y_test)
        
        accuracies.append(accuracy)
        
    return np.mean(accuracies)

# Tuning Hyperparameters

In [7]:
from joblib import Parallel, delayed


lambda_values = [1e-5, 1e-6, 1e-7]

# Gaussian kernel parameters:
gamma_values = [0.1, 1, 10]
max_iter_values = [100, 1000, 10000]

# Polynomial kernel parameters
degree_values = [2, 3, 4]
coef0_values = [0.1, 1, 10]

inputData = trainData[: int(0.4 * len(trainData))]
outputData = trainLabel[: int(0.4 * len(trainLabel))]

def evaluate_params(params):
    mean_loss = crossValScore(inputData, outputData, params, k=3)
    return params, mean_loss

# Gaussian kernel
print("-" * 60)
print("Tuning Hyperparameters for Gaussian Kernel: \n")
best_gaussian_loss = 2
best_gaussian_params = {}

# Generate parameter combinations for Gaussian kernel
gaussian_params_list = [{'kernel': 'gaussian', '_lambda': _lambda, 'gamma': gamma, 'degree': 3, 'coef0': 1.0, 'max_iter': iters}
                        for gamma in gamma_values 
                        for iters in max_iter_values 
                        for _lambda in lambda_values]

# Evaluate all parameter combinations in parallel
results = Parallel(n_jobs=-1)(delayed(evaluate_params)(params) for params in gaussian_params_list)

for params, mean_loss in results:
    print(f"\nlambda: {params['_lambda']}, gamma: {params['gamma']}, n_iters: {params['max_iter']}, \n\t\t\t\tCross Validation Loss:{mean_loss:.4f}")
    if mean_loss < best_gaussian_loss:
        best_gaussian_loss = mean_loss
        best_gaussian_params = params

print("\nBest Hyperparameters:\n")
print("Gaussian: ")
print(f"\t lambda: {best_gaussian_params['_lambda']}")
print(f"\t gamma: {best_gaussian_params['gamma']}")
print(f"\t max_iters: {best_gaussian_params['max_iter']}")
print(f"\t Best Cross-Validation loss: {best_gaussian_loss}")

# Polynomial Kernel
print("-" * 60)
print("Tuning Hyperparameters for Polynomial Kernel: \n")
best_polynomial_loss = 2
best_polynomial_params = {} 

# Generate parameter combinations for Polynomial kernel
polynomial_params_list = [{'kernel': 'polynomial', '_lambda': _lambda, 'gamma': gamma, 'degree': degree, 'coef0': coef0, 'max_iter': iters}
                          for degree in degree_values 
                          for coef0 in coef0_values
                          for gamma in gamma_values 
                          for iters in max_iter_values
                          for _lambda in lambda_values]

# Evaluate all parameter combinations in parallel
results = Parallel(n_jobs=-1)(delayed(evaluate_params)(params) for params in polynomial_params_list)

for params, mean_loss in results:
    print(f"\ndegree: {params['degree']}, coef0: {params['coef0']}, gamma: {params['gamma']}, n_iters: {params['max_iter']}, \n\t\t\t\tCross Validation Loss:{mean_loss:.4f}")
    if mean_loss < best_polynomial_loss:
        best_polynomial_loss = mean_loss
        best_polynomial_params = params

print("-" * 60)

print("\nPolynomial: ")
print(f"\t lambda: {best_polynomial_params['_lambda']}")
print(f"\t degree: {best_polynomial_params['degree']}")
print(f"\t gamma: {best_polynomial_params['gamma']}")
print(f"\t coef0: {best_polynomial_params['coef0']}")
print(f"\t max_iters: {best_polynomial_params['max_iter']}")
print(f"\t Best Cross-Validation loss: {best_polynomial_loss}")

------------------------------------------------------------
Tuning Hyperparameters for Gaussian Kernel: 


lambda: 1e-05, gamma: 0.1, n_iters: 100, 
				Cross Validation Loss:0.2927

lambda: 1e-06, gamma: 0.1, n_iters: 100, 
				Cross Validation Loss:0.3215

lambda: 1e-07, gamma: 0.1, n_iters: 100, 
				Cross Validation Loss:0.3765

lambda: 1e-05, gamma: 0.1, n_iters: 1000, 
				Cross Validation Loss:0.3146

lambda: 1e-06, gamma: 0.1, n_iters: 1000, 
				Cross Validation Loss:0.2933

lambda: 1e-07, gamma: 0.1, n_iters: 1000, 
				Cross Validation Loss:0.2695

lambda: 1e-05, gamma: 0.1, n_iters: 10000, 
				Cross Validation Loss:0.2980

lambda: 1e-06, gamma: 0.1, n_iters: 10000, 
				Cross Validation Loss:0.2689

lambda: 1e-07, gamma: 0.1, n_iters: 10000, 
				Cross Validation Loss:0.2724

lambda: 1e-05, gamma: 1, n_iters: 100, 
				Cross Validation Loss:0.2817

lambda: 1e-06, gamma: 1, n_iters: 100, 
				Cross Validation Loss:0.3243

lambda: 1e-07, gamma: 1, n_iters: 100, 
				Cross V

In [8]:
best_gaussian_params

{'kernel': 'gaussian',
 '_lambda': 1e-05,
 'gamma': 10,
 'degree': 3,
 'coef0': 1.0,
 'max_iter': 10000}

In [9]:
params = best_gaussian_params

kernelPegasosSVM = KernelPegasosSVM(kernel = params['kernel'], _lambda = params['_lambda'], gamma = params['gamma'], degree =  params['degree'], coef0 = params['coef0'], max_iter =  params['max_iter'])
kernelPegasosSVM.fit(trainData, trainLabel)


preds = kernelPegasosSVM.predict(testData)
zeroOneLoss(preds, testLabel)

0.159

In [10]:
params = best_polynomial_params

kernelPegasosSVM = KernelPegasosSVM(kernel = params['kernel'], _lambda = params['_lambda'], gamma = params['gamma'], degree =  params['degree'], coef0 = params['coef0'], max_iter =  params['max_iter'])
kernelPegasosSVM.fit(trainData, trainLabel)

preds = kernelPegasosSVM.predict(testData)
zeroOneLoss(preds, testLabel)

0.2155

In [11]:
best_polynomial_params

{'kernel': 'polynomial',
 '_lambda': 1e-05,
 'gamma': 1,
 'degree': 4,
 'coef0': 1,
 'max_iter': 10000}